# --- Day 12: Hill Climbing Algorithm ---

You try contacting the Elves using your handheld device, but the river you're following must be too low to get a decent signal.

You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, where a is the lowest elevation, b is the next-lowest, and so on up to the highest elevation, z.

Also included on the heightmap are marks for your current position (S) and the location that should get the best signal (E). Your current position (S) has elevation a, and the location that should get the best signal (E) has elevation z.

You'd like to reach E, but to save energy, you should do it in as few steps as possible. During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be at most one higher than the elevation of your current square; that is, if your current elevation is m, you could step to elevation n, but not to elevation o. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)

For example:
```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```
Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward the e at the bottom. From there, you can spiral around to the goal:
```
v..v<<<<
>v.vv<<^
.>vv>E^^
..v>>>^^
..>>>>>^
```
In the above diagram, the symbols indicate whether the path exits each square moving up (^), down (v), left (<), or right (>). The location that should get the best signal is still E, and . marks unvisited squares.

This path reaches the goal in 31 steps, the fewest possible.

What is the fewest steps required to move from your current position to the location that should get the best signal?

In [ ]:
import logging
import sys

sys.path.append("..")
from utilsp import get_logger, get_data, timer, test

logger = get_logger("day12", logging.INFO)
data = get_data("day12")

logger.info("initialization complete!")

## Solution Part One
Instead of writing a pathfinding algorithm, I'm just going to convert the mountain matrix to a one-directional graph, and let the networkx library do the rest.

In [ ]:
import networkx as nx


def get_start_end(matrix: list):
    """Find the S and E in the input matrix"""
    start, end = None, None
    for y, row in enumerate(matrix):
        for x, val in enumerate(row):
            if val == "S":
                start = (x, y)
            elif val == "E":
                end = (x, y)
    return start, end


class Graph(object):
    def __init__(self, input_: list, start=None, end=None):
        self.matrix = [
            [
                ord(c) - ord("a") + 1
                if c.islower()
                else 0
                if c == "S"
                else 27
                if c == "E"
                else c
                for c in line
            ]
            for line in input_
        ]
        self.start = start
        self.end = end
        self.graph = nx.DiGraph()
        self._add_nodes_and_edges()

    def get_neighbors(self, node):
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        neighbors = []
        for direction in directions:
            neighbor = (node[0] + direction[0], node[1] + direction[1])
            if neighbor in self.graph.nodes:
                neighbors.append(neighbor)
        return neighbors

    def _add_nodes_and_edges(self):
        for y in range(len(self.matrix)):
            for x in range(len(self.matrix[y])):
                self.graph.add_node((x, y), height=self.matrix[y][x])

        for n in self.graph.nodes:
            for neighbour in self.get_neighbors(n):
                if (
                    self.graph.nodes[neighbour]["height"]
                    - self.graph.nodes[n]["height"]
                    <= 1
                ):
                    self.graph.add_edge(n, neighbour)

    def get_shortest_path(self):
        return nx.shortest_path(self.graph, self.start, self.end)


@timer
def part1(input_: str):
    input_ = input_.split()
    start, end = get_start_end(input_)
    graph = Graph(input_, start, end)
    path = graph.get_shortest_path()
    return len(path) - 1


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 1:", part1(data.input))

# --- Part Two ---

As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.

To maximize exercise while hiking, the trail should start as low as possible: elevation a. The goal is still the square marked E. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from any square at elevation a to the square marked E.

Again consider the example from above:
```
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
```
Now, there are six choices for starting position (five marked a, plus the square marked S that counts as being at elevation a). If you start at the bottom-left square, you can reach the goal most quickly:
```
...v<<<<
...vv<<^
...v>E^^
.>v>>>^^
>^>>>>>^
```
This path reaches the goal in only 29 steps, the fewest possible.

What is the fewest steps required to move starting from any square with elevation a to the location that should get the best signal?

## Solution Part Two

In [ ]:
class Graph2(Graph):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_lowest(self):
        """returns all nodes with a height of a (1)"""
        return [n for n in self.graph.nodes if self.graph.nodes[n]["height"] == 1]


@timer
def part2(input_):
    input_ = input_.split("\n")
    _, end = get_start_end(input_)
    graph = Graph2(input_, end=end)
    lowest = graph.get_lowest()
    paths = []
    for node in lowest:
        graph.start = node
        try:
            path = graph.get_shortest_path()
            paths.append(path)
        except nx.exception.NetworkXNoPath:
            pass
    return len(min(paths, key=len)) - 1


test(func=part2, input_=data.example_input, expected=data.test_answer2)

print("solution part 2:", part2(data.input))